**Capstone Code: Section 3, Merge Data**

In [1]:
# load libraries and other required methods

import numpy as np                                                       # library to handle data in a vectorized manner
from numpy.random import randint 
import random                                                            # library for random number generation
import requests                                                          # library to handle requests

import pandas as pd                                                      # library for data analsysis

import json                                                              # library to handle JSON files
from pandas.io.json import json_normalize                                # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim                                    # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="foursquare_agent")

import matplotlib.cm as cm                                               # Matplotlib and associated plotting modules
import matplotlib.colors as colors                                       # Matplotlib and associated plotting modules
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans                                       # import k-means from clustering stage
from sklearn.linear_model import LinearRegression

from IPython.display import Image                                        # libraries for displaying images
from IPython.core.display import HTML                                    # libraries for displaying images
import folium                                                            # map rendering library

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                          # display requested dataframes in pandas format

print('Libraries imported.\n')

Libraries imported.



**Read Michigan city and restaurant data, left join into dataframe, sort by cities, save files to disk**

*The following city and restaurant source data was obtained from a notebook located at https://github.com/bikerhino/Capstone/blob/main/cities.ipynb.

In [2]:
cities2 = pd.read_csv(r'C:\Users\Jim_2\Desktop\python_projects\cities2.csv')              # reads cities2 file data
cities2.index = np.arange(1, len(cities2)+ 1 )                                            # resets cities2 index to 1
capMex = pd.read_csv(r'C:\Users\Jim_2\Desktop\python_projects\capMex.csv')                # read restaurant csv into a dataframe
mexReviews = pd.read_csv(r'C:\Users\Jim_2\Desktop\python_projects\mexReviews.csv')        # read restaurant review csv into a dataframe

capMex2 = capMex.merge(mexReviews, on=['id','name'], how='left')                          # merge mex restaurants and review data
r_mean = mexReviews['rating'].mean(axis=0)
l_mean = mexReviews['likes'].mean(axis=0)
r_mean, l_mean = round(r_mean, 1), round(l_mean, 1)
print ('\nThe average rating for all found restaurants is: ',r_mean)
print ('The average likes for all found restaurants is: ',l_mean,'\n')

for c, row in cities2.iterrows():                                          # loop through cities2 to save city retaurant dataframe
    city = row['city']                                                     # declare city variable
    
    cityMex = capMex2.loc[capMex2['city'] == city ]                        # assign dataframe by city
    cityChar3 = city[:3]                                                   # assign variable to first 3 city characters
    onlyCity2 = '_only%s'%cityChar3                                        # combine string with prior variable as a new variable
    cityurl2 = 'C:\\Users\Jim_2\Desktop\python_projects\cities\{}.csv'.format(onlyCity2)   # create url for each city
    cityMex.to_csv(cityurl2, index=False, header=True)                     # saving file to disk
    
    cities2.loc[c,'avg_rating'] = round(cityMex['rating'].mean(axis=0),1)
    cities2.loc[c,'avg_rating_no0'] = round(cityMex.loc[cityMex['rating'] !=0,'rating'].mean(axis=0),1)
    cities2.loc[c,'avg_likes'] = round(cityMex['likes'].mean(axis=0),1)
    cities2.loc[c,'avg_likes_no0'] = round(cityMex.loc[cityMex['likes'] !=0,'likes'].mean(axis=0),1)

cities2['rest_density'] = round(cities2.apply(lambda x: x['population']/x['count'], axis=1),1)

capMex2 = capMex2.fillna(0)
capMex2.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\capMex2.csv', index=False, header=True) # saving file to disk
cities2 = cities2.rename(columns = {"lat":"city_lat","lng":"city_lng"}) 
cities2.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\cities2.csv', index=False, header=True) # saving updated cities2 to disk

maxMex = capMex2.merge(cities2, on=['city'], how='left')                          # merge mex restaurants and review data

maxMex['rating_diff'] = round(maxMex['rating']- maxMex['avg_rating'], 1)
maxMex['likes_diff'] = round(maxMex['likes']- maxMex['avg_likes'], 1)

maxMex = maxMex[['name','city','rating','likes','tier','rest_density','rating_diff','likes_diff','growth_rate']].copy()
maxMex.head()
maxMex.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\maxMex.csv', index=False, header=True) # saving updated maxMex to disk


The average rating for all found restaurants is:  5.4
The average likes for all found restaurants is:  28.5 



,name,city,rating,likes,tier,rest_density,rating_diff,likes_diff,growth_rate
0,Mexican Town Restaurant,Detroit,7.2,40,1,70122.5,1.3,1.6,0.005
1,Mexican Village Restaurant,Detroit,8.1,116,2,70122.5,2.2,77.6,0.005
2,Qdoba Mexican Grill,Detroit,7.4,34,1,70122.5,1.5,-4.4,0.005
3,On The Border Mexican Grill & Cantina,Detroit,7.2,88,2,70122.5,1.3,49.6,0.005
4,Mexican Gardens,Detroit,8.2,59,2,70122.5,2.3,20.6,0.005
